# In this notebook we will create a background changer for videos with a condition that these videos have a greenscreen background
We will use Numpy, OpenCv, MatplotLib and MoviePy. For the ones who doesn't know MoviePy is a Python module for video editing.
We will use another video as a background.
In this notebook we will not only change the background of a video with a static image, we will change the background of the video with another video.

### Importing the resources

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import moviepy.editor as movie

### Remeber OpenCv shape attribute return the size as (height, width, depth) and MoviePy size attribute return (width, height) and the OpenCv resize function take a tuple argumet of height and width as (width, height)

In [ ]:
#import the video
clip = movie.VideoFileClip("videos/van_damme_greenscreen.mp4").subclip(10, 50)

#print the size of th video
print("Video size: ", clip.size)

#import the background video
background_clip = movie.VideoFileClip("videos/new_york_street.mp4").subclip(10, 50)

#print the size of the background video
print("background video size: ", background_clip.size)

In [ ]:
#print the fps of the clip
print("clip fps: ", clip.fps)

#print fps of background clip
print("background fps clip: ", background_clip.fps)

In [ ]:
#make a copy of the clip
clip_copy = clip.copy()

#make a copy of background clip
background_clip_copy = background_clip.copy()

In [ ]:
#extracting the audio soundtrack of the clip
audioclip = clip.audio

In [ ]:
audioclip = audioclip.set_duration(40)
audioclip = audioclip.set_start(10, change_end=True)

# We are going to generate a sequence of images from the video, change the background of each farme with his pairing frame from another video, save the new images as jpg in a folder named image_sequence
We will use a video as background for our video that we intend to change its background.

In [ ]:
#get the lenght of the clip
video_length = sum((1 for img in clip_copy.iter_frames()))

#get the lenght of the background clip
#background_clip_lenght = sum((1 for img in background_clip_copy.iter_frames()))

In [ ]:
#creating the list of frame names for the video that we will generate
full_image_names = []
for i in range(video_length):
    full_image_names.append(f"frame_{i}.jpg")

In [ ]:
#sort the list of names to make it is input for the imageSequenceClip
full_image_names = sorted(full_image_names)

## Generate the sequence of images by pairing the frames that correspond to each other from each video

In [ ]:
for i, (frame, background_frame) in enumerate(zip(clip_copy.iter_frames(dtype = "uint8"), background_clip_copy.iter_frames(dtype = "uint8")), 0):
#for i, frame in enumerate(clip_copy.iter_frames(dtype = "uint8"), 0):
    #make a copy of the current frame
    imageHsv = np.copy(frame)

    #make a copy of the current background frame
    backgroundImgHsv = np.copy(background_frame)

    #now let's change the color space from RGB to HSV
    #remeber MoviePy use RGB color space and the video is imported with MoviePy
    imageHsv = cv2.cvtColor(imageHsv, cv2.COLOR_RGB2HSV)
    backgroundImgHsv = cv2.cvtColor(backgroundImgHsv, cv2.COLOR_RGB2HSV)

    #defining the mask boundaries
    lowerMaskH = np.array([58, 0, 0])
    upperMaskH = np.array([90, 255, 255])

    #define the masked area
    maskedArea = cv2.inRange(imageHsv, lowerMaskH, upperMaskH)

    #mask the current frame
    maskedImageHsv = np.copy(imageHsv)
    maskedImageHsv[maskedArea != 0] = [0, 0, 0]

    #mask the background image
    maskedBackgroundImgHsv = np.copy(backgroundImgHsv)
    maskedBackgroundImgHsv[maskedArea == 0] = [0, 0, 0]

    #and now merge the two images together 
    finalImage = maskedImageHsv + maskedBackgroundImgHsv

    #convert the final image from HSV to BGR
    final_frame = cv2.cvtColor(finalImage, cv2.COLOR_HSV2BGR)
    
    #write the curret frame to the hard disk
    #reneber we need the color space to be BGR for OpenCv
    cv2.imwrite(f"image_sequence_v/{full_image_names[i]}", final_frame)

## Importing the image sequence to a clip and writing it to a video file

In [ ]:
#importing the imgae sequence to a clip
image_sequence_clip = movie.ImageSequenceClip("image_sequence_v/", fps = 30)

In [ ]:
#setting the audioclip as a soundtrack of this new imported image sequence clip
image_sequence_clip = image_sequence_clip.set_audio(audioclip)

In [ ]:
#writing the imageSequenceClip clip to a video file
image_sequence_clip.write_videofile("videos/background_changed_vandamme_with_video_as_background.mp4")